# filter()
- `DataFrameGroupBy.filter(func, dropna=True, *args, **kwargs)`
- 특정 조건을 만족하는 Group의 데이터(행)들을 조회할 때 사용한다. 주로 조건은 group별 집계결과를 이용한다.
    1. 함수에 group별 DataFrame을 argument로 전달한다.
    2. 함수는 받은 DataFrame을 이용해 집계한 값의 조건을 비교해서 반환한다.(반환타입: Bool) 
    3. 반환값이 True인 Group들의 모든 행들로 구성된 DataFrame을 반환한다.
- 매개변수
    - **func**: filtering 조건을 구현한 함수 객체
        - 첫번째 매개변수로 Group으로 묶인 DataFrame을 받는다.
        - bool type 값을 반환한다. 매개변수로 받은 DataFrame이 특정 조건을 만족하는지 여부를 반환한다.
    - **dropna=True**
        - 필터를 통과하지 못한 group의 DataFrame의 값들을 drop시킨다(기본값). False로 설정하면 NA 처리해서 반환한다.
    - **\*args, \*\*kwargs**: filter 함수의 두번째부터  선언된 매개변수에 전달할 argument 값들을 가변인자로 전달한다.

In [1]:
import numpy as np
import pandas as pd

# cnt1 값의 범위: 사과: 10대, 귤: 20대, 배: 단단위, 딸기 30이상
data = dict(fruits=['사과', '사과','사과', '사과','사과','귤','귤','귤','귤','귤','배','배','배','배','배','딸기','딸기','딸기','딸기','딸기']
            ,cnt1=[10, 12, 13, 11, 12, 21, 22, 27, 24, 26, 7, 7, 8, 3, 2, 30, 35, 37, 41, 28]
            ,cnt2=[100, 103, 107, 107,  101,  51,  57, 58,  57, 51,  9, 9,  5, 7,  7, 208, 217, 213, 206, 204]
           )
df = pd.DataFrame(data)
df

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


In [4]:
# filter 함수 - cnt1의 평균이 20이상인 과일 데이터들을 조회
result = df.groupby('fruits')['cnt1'].mean()
result2 = result[result >= 20]
df[df['fruits'].isin(result2.index)]

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [5]:
def check_mean(df):
    """
    df - DataFrame. filter에 사용하면 Group별로 DataFrame이 넘어온다.
    """
    return df['cnt1'].mean() >= 20

In [6]:
df.groupby('fruits').filter(check_mean)  # 함수가 그룹별로 4번 호출

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [9]:
# lambda 표현식: return하는 구문까지 한 줄일 때 => 바로 return일 때
df.groupby('fruits').filter(lambda df : df['cnt1'].mean() > 20) # Return 결과가 True인 group의 dataframe의 결과에 추가.

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [11]:
# Parameter가 있는 filter처리함수
# 컬럼명, threshold 값을 parameter로 받는다
def check_mean2(df:pd.DataFrame, column_name:str, threshold:int):
    return df[column_name].mean() >= threshold

In [17]:
# 1번 parameter - group 별 DataFrame이 전달. 2번째 parameter 부터는 남겨줘야한다.
df.groupby('fruits').filter(check_mean2, column_name = 'cnt1', threshold = 30)
df.groupby('fruits').filter(check_mean2, column_name = 'cnt2', threshold = 100)
df.groupby('fruits').filter(check_mean2, dropna=False, column_name = 'cnt2', threshold = 100) # dropna: False인 부분은 모두 결측치
# 람다 식은 필요한 값을 다 넣기 때문에 굳이 사용하지 않는다.

,fruits,cnt1,cnt2
0,사과,10.0,100.0
1,사과,12.0,103.0
2,사과,13.0,107.0
3,사과,11.0,107.0
4,사과,12.0,101.0
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


# transform
- `DataFrameGroupBy.transform(func, *args)`, `SeriesGroupBy.transform(func, *args)`
    - 함수(func)에 열의 값들을 group 별로 전달 한다. 함수는 그 값을 받아 통계량을 구해 반환한다. 반환된 통계량으로 원래 값들을 변경한 Series를 반환한다. 여러 컬럼에 대해 처리할 경우 DataFrame을 반환한다.
    - func: 함수객체
        - 매개변수
            - 그룹별 컬럼값들을 받을 변수 선언
        - return
            - 계산한 통계량.
        - DataFrameGroupBy은 모든 컬럼의 값들을 group 별 Series로 전달한다.
    - *args: 함수에 전달할 추가 인자값이 있으면 매개변수 순서에 맞게 값을 전달한다.
- transform() 함수를 groupby() 와 사용하면 컬럼의 각 원소들을 자신이 속한 그룹의 통계량으로 변환된 데이터셋을 생성할 수 있다.
- 컬럼의 값과 통계값을 비교해서 보거나 결측치 처리등에 사용할 수있다.

## 원본에 통계치 붙여서 비교하기

In [19]:
# DataFrameGroupby -> group별로 각 컬럼의 값을 함수에 전달.
# 사과 -> cnt1, 귤 -> cnt1, 사과 -> cnt2, 귤 -> cnt2 ...
df.groupby('fruits').transform("mean")

,cnt1,cnt2
0,11.6,103.6
1,11.6,103.6
2,11.6,103.6
3,11.6,103.6
4,11.6,103.6
5,24.0,54.8
6,24.0,54.8
7,24.0,54.8
8,24.0,54.8
9,24.0,54.8


In [20]:
df.groupby('fruits')['cnt1'].transform("mean")

0     11.6
1     11.6
2     11.6
3     11.6
4     11.6
5     24.0
6     24.0
7     24.0
8     24.0
9     24.0
10     5.4
11     5.4
12     5.4
13     5.4
14     5.4
15    34.2
16    34.2
17    34.2
18    34.2
19    34.2
Name: cnt1, dtype: float64

In [22]:
result = df.groupby('fruits').transform('mean')
result

,cnt1,cnt2
0,11.6,103.6
1,11.6,103.6
2,11.6,103.6
3,11.6,103.6
4,11.6,103.6
5,24.0,54.8
6,24.0,54.8
7,24.0,54.8
8,24.0,54.8
9,24.0,54.8


In [23]:
# 컬럼(열) 삽입 - df.insert(삽입할 컬럼 순번, 컬럼 이름, 값들)
df.insert(1, "cnt1평균", result['cnt1'])
df

,fruits,cnt1평균,cnt1,cnt2
0,사과,11.6,10,100
1,사과,11.6,12,103
2,사과,11.6,13,107
3,사과,11.6,11,107
4,사과,11.6,12,101
5,귤,24.0,21,51
6,귤,24.0,22,57
7,귤,24.0,27,58
8,귤,24.0,24,57
9,귤,24.0,26,51


In [24]:
df['cnt2평균'] = result['cnt2']
df

,fruits,cnt1평균,cnt1,cnt2,cnt2평균
0,사과,11.6,10,100,103.6
1,사과,11.6,12,103,103.6
2,사과,11.6,13,107,103.6
3,사과,11.6,11,107,103.6
4,사과,11.6,12,101,103.6
5,귤,24.0,21,51,54.8
6,귤,24.0,22,57,54.8
7,귤,24.0,27,58,54.8
8,귤,24.0,24,57,54.8
9,귤,24.0,26,51,54.8


In [36]:
df2 = pd.DataFrame(data)
df2.sample(n=3)  # random 하게 행을 추출

,fruits,cnt1,cnt2
6,귤,22,57
8,귤,24,57
16,딸기,35,217


In [37]:
df2.sample(frac=0.3) # 비율 (전체중 30%의 행만 추출)

,fruits,cnt1,cnt2
17,딸기,37,213
10,배,7,9
11,배,7,9
3,사과,11,107
8,귤,24,57
6,귤,22,57


In [38]:
df2.sample(frac=1) # frac = 1 => 행 섞기

,fruits,cnt1,cnt2
10,배,7,9
6,귤,22,57
2,사과,13,107
15,딸기,30,208
0,사과,10,100
9,귤,26,51
14,배,2,7
11,배,7,9
8,귤,24,57
12,배,8,5


In [41]:
df2.reset_index(drop=True, inplace=True) # drop=True : 이전 index 값 지우고 원래대로 되돌리기

In [44]:
cnt1_avg = df2.groupby('fruits')['cnt1'].transform('mean')
df2.insert(2, "과일별 cnt1평균", cnt1_avg)

In [45]:
df2

,fruits,cnt1,과일별 cnt1평균,cnt2
0,사과,10,11.6,100
1,사과,12,11.6,103
2,사과,13,11.6,107
3,사과,11,11.6,107
4,사과,12,11.6,101
5,귤,21,24.0,51
6,귤,22,24.0,57
7,귤,27,24.0,58
8,귤,24,24.0,57
9,귤,26,24.0,51


## 결측치 처리
- transform이용해서 결측치를 같은 과일별 평균값으로 변환
    - 전체 평균보다 좀더 정확할 수 있다.

In [51]:
import numpy as np
s = pd.Series([1, np.nan, 2, np.nan])
s

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [52]:
# 결측치(nan)을 다른 값을 대체(변경)
s.fillna(20) #모든 결측치값을 20으로 변경

0     1.0
1    20.0
2     2.0
3    20.0
dtype: float64

In [ ]:
# s.fillna(s와 동일한 크기의 Series) -> 결측치는 결측치와 같은 index의 값으로 변경. 결측치 아닌 것들은 유지.

In [55]:
s.fillna(pd.Series([100, 200, 300, 400]))

0      1.0
1    200.0
2      2.0
3    400.0
dtype: float64

In [76]:
df3 = pd.DataFrame(data)
df3.loc[[0, 1, 5, 6, 10, 11, 15, 16], 'cnt2'] = np.nan
df3   # 결측치 바꾸기

,fruits,cnt1,cnt2
0,사과,10,NaN
1,사과,12,NaN
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
5,귤,21,NaN
6,귤,22,NaN
7,귤,27,58.0
8,귤,24,57.0
9,귤,26,51.0


In [59]:
df3.dropna(axis=1) 
# DataFrame에서 결측치를 제거 -> 행 제거, axis = 1 하면 칼럼(열)을 제거

,fruits,cnt1
0,사과,10
1,사과,12
2,사과,13
3,사과,11
4,사과,12
5,귤,21
6,귤,22
7,귤,27
8,귤,24
9,귤,26


In [61]:
cnt2_mean = df3['cnt2'].mean()
cnt2_mean

93.58333333333333

In [70]:
df3['cnt2'] = df3['cnt2'].fillna(round(cnt2_mean))
df3

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


In [77]:
df3 = pd.DataFrame(data)
df3['cnt2'] = df3['cnt2'].fillna(round(cnt2_mean))
df3

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


In [78]:
df3['cnt2'] = df3['cnt2'].fillna(round(df3.groupby('fruits')['cnt2'].transform('mean')))
df3

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


<b style='font-size:2em'>TODO </b>

In [21]:
# 1.  data/diamonds.csv 조회
import pandas as pd
dia = pd.read_csv("data/diamonds.csv")
dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [2]:
# 2.  cut 별 평균 가격이 4000 이상인 diamond 데이터들 조회 
# cut별 평균가격
result = dia.groupby('cut')['price'].mean()
# 4000 이상인것
result[result >= 4000]

cut
Fair       4358.757764
Premium    4584.257704
Name: price, dtype: float64

In [5]:
result = dia.groupby('cut').filter(lambda x : x['price'].mean() >= 4000)
result['cut'].value_counts()

Premium    13791
Fair        1610
Name: cut, dtype: int64

In [9]:
def check_mean(x):
    return x['price'].mean() >= 4000

dia.groupby('cut').filter(check_mean)['cut'].value_counts()

Premium    13791
Fair        1610
Name: cut, dtype: int64

In [12]:
# 3. color 별 carat의 최대값과 최소값의 차이가 2이상 3미만인 모든 diamond 데이터들 조회
def carat_min_max_diff(dia):
    sub = dia['carat'].max() - dia['carat'].min()
    return (sub >= 2) & (sub < 3)

result = dia.groupby('color').filter(carat_min_max_diff)
# result
result['color'].value_counts()

G    11292
E     9797
F     9542
Name: color, dtype: int64

In [14]:
dia.groupby('color')['carat'].agg(lambda x : x.max() - x.min())

color
D    3.20
E    2.85
F    2.81
G    2.78
H    3.90
I    3.78
J    4.78
Name: carat, dtype: float64

In [22]:
# 4. clarity 별 평균 가격 컬럼을 DataFrame에 추가
mean_price = dia.groupby('clarity')['price'].transform('mean')
# dia['clarity 별 평균 가격'] = round(mean_price) # 맨 끝에 추가하기
dia.insert(7, 'clarity means', mean_price)
dia

,carat,cut,color,clarity,depth,table,price,clarity means,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,5063.028606,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3996.001148,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,3839.455391,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,3924.989395,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,5063.028606,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,3996.001148,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,3996.001148,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,3996.001148,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,5063.028606,6.15,6.12,3.74


# pivot_table()
엑셀의 pivot table 기능을 제공하는 메소드.    
분류별 집계(Group으로 묶어 집계)를 처리하는 함수로 group으로 묶고자 하는 컬럼들을 행과 열로 위치시키고 집계값을 값으로 보여준다.    
역할은 `groupby()`를 이용한 집계와 같은데 **여러개 컬럼을 기준으로 groupby 를 할 경우 집계결과를 읽는 것이 더 편하다.(가독성이 좋다)**

> pivot() 함수와 역할이 다르다.   
> pivot() 은 index와 column의 형태를 바꾸는 reshape 함수.

- `DataFrame.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')`
- **매개변수**
    - **index**
        - 문자열 또는 리스트. index로 올 컬럼들 => groupby였으면 묶었을 컬럼
    - **columns**
        - 문자열 또는 리스트. column으로 올 컬럼들 => groupby였으면 묶었을 컬럼 (index/columns가 묶여서 groupby에 묶을 컬럼들이 된다.)
    - **values**
        - 문자열 또는 리스트. 집계할 대상 컬럼들
    - **aggfunc**
        - 집계함수 지정. 함수, 함수이름문자열, 함수리스트(함수이름 문자열/함수객체), dict: 집계할 함수
        - 기본(생략시): 평균을 구한다. (mean이 기본값)
    - **fill_value, dropna**
        - fill_value: 집계시 NA가 나올경우 채울 값
        - dropna: boolean. 컬럼의 전체값이 NA인 경우 그 컬럼 제거(기본: True)
    - **margins/margins_name**
        - margin: boolean(기본: False). 총집계결과를 만들지 여부.
        - margin_name: margin의 이름 문자열로 지정 (생략시 All)

In [86]:
flights = pd.read_csv('data/flights.csv')

In [88]:
flights.groupby(['ORG_AIR', 'AIRLINE', 'MONTH'])['DEP_DELAY'].mean()  # groupby으로 집계한 결과가 너무 많다 ->pivot_table

ORG_AIR  AIRLINE  MONTH
ATL      AA       1        -3.250000
                  2        -3.000000
                  3        -0.166667
                  4         0.071429
                  5         5.777778
                             ...    
SFO      WN       7         7.578947
                  8         8.535714
                  9         5.921053
                  11        9.500000
                  12       26.589744
Name: DEP_DELAY, Length: 1133, dtype: float64

## 1개의 컬럼을 grouping 해서 집계
- 항공사별 비행시간의 평균 
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
    - 집계대상컬럼
        - AIR_TIME
- 집계: mean

In [103]:
# flight.groupby('AIRLINE')['AIR_TIME'].mean().to_frame()  # Series -> DataFrame
flights.pivot_table(index='AIRLINE', values='AIR_TIME', margins=True, margins_name="총계") # 반환: DataFrame # aggfunc의 Default 값: mean

,AIR_TIME
AIRLINE,
AA,144.259404
AS,147.845052
B6,209.412963
DL,115.334187
EV,68.964016
F9,127.592337
HA,338.288288
MQ,61.318346
NK,135.736878


In [96]:
# 결과 값을 table 형태로 보여줘 보기 쉽게 만든다
flights.pivot_table(index='ORG_AIR', columns='AIRLINE', values='DEP_DELAY', aggfunc='mean')

AIRLINE,AA,AS,B6,DL,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
ORG_AIR,,,,,,,,,,,,,,
ATL,7.995652,15.307692,NaN,7.616505,10.140896,21.574324,NaN,23.215686,25.949153,10.645390,15.279070,2.281250,NaN,13.232085
DEN,10.367442,-2.600000,16.576923,2.734513,15.265823,10.461295,NaN,NaN,10.793388,8.790047,14.520028,1.169231,NaN,11.631183
DFW,11.408569,2.723404,8.333333,14.550000,12.279942,22.095238,NaN,10.285337,14.334601,24.057143,16.500000,5.707547,NaN,NaN
IAH,13.046632,-6.857143,NaN,11.707071,4.528571,7.909091,NaN,12.500000,26.673759,14.885057,17.818735,-1.135135,NaN,NaN
LAS,10.347709,2.909091,15.557377,2.970100,NaN,10.558559,4.545455,NaN,15.325949,-0.716216,13.522989,-2.790698,6.224299,12.363636
LAX,8.301848,-0.679389,9.787402,9.572988,NaN,12.842105,4.960000,21.833333,26.894309,10.002681,13.706535,4.178571,9.276243,15.697464
MSP,5.062500,5.933333,NaN,5.279908,15.823232,36.333333,NaN,49.000000,18.876289,5.089636,16.435897,0.486111,NaN,8.444915
ORD,16.450444,8.192308,24.273973,11.054054,10.155657,20.637037,NaN,11.315639,25.373188,11.901751,20.574896,6.525773,21.804878,NaN
PHX,8.183790,-4.515625,27.631579,2.598214,4.000000,14.777778,-1.052632,NaN,6.605263,4.203704,7.508671,5.812941,NaN,12.444121


In [104]:
flights.pivot_table(columns='AIRLINE', values='AIR_TIME') # aggfunc의 Default 값: mean

AIRLINE,AA,AS,B6,DL,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
AIR_TIME,144.259404,147.845052,209.412963,115.334187,68.964016,127.592337,338.288288,61.318346,135.736878,76.010272,155.650521,147.686755,154.864097,107.005897


## 두개의 컬럼을 grouping 해서 집계
- 항공사/출발공항코드 별 취소 총수 (1이 취소이므로 합계를 구한다.)
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
        - ORG_AIR: 출발 공항코드
    - 집계대상컬럼
        - CANCELLED: 취소여부 - 1:취소, 0: 취소안됨
- 집계: sum

In [107]:
flights.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum().to_frame()  # 있는 값만 들어가기 때문에 결측치가 들어가지 X

CANCELLED
AIRLINE ORG_AIR           
AA      ATL              3
        DEN              4
        DFW             86
        IAH              3
        LAS              3
...                    ...
WN      LAS              7
        LAX             32
        MSP              1
        PHX              6
        SFO             25

[114 rows x 1 columns]

In [109]:
flights.pivot_table(index='AIRLINE', columns='ORG_AIR', values='CANCELLED', aggfunc='sum', fill_value=-9999, margins=True
                   ) # 결측치를 대체할 값. (결측치- 그 그룹으로 묶인행이 없음)
                     # 값들의 조합이기 때문에 결측치가 나온다.
                     # 결측치의 값을 지정해도 margins은 계산하지 않는다. 

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO,All
AIRLINE,,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2,154
AS,0,0,0,0,0,0,0,0,0,0,0
B6,-9999,0,0,-9999,0,0,-9999,0,0,1,1
DL,28,1,0,0,1,1,4,0,1,2,38
EV,18,6,27,36,-9999,-9999,6,53,0,-9999,146
F9,0,2,1,0,1,1,1,4,0,0,10
HA,-9999,-9999,-9999,-9999,0,0,-9999,-9999,0,0,0
MQ,5,-9999,62,0,-9999,0,0,85,-9999,-9999,152
NK,1,1,6,0,1,1,3,10,2,-9999,25


## 3개 이상의 컬럼을 grouping해서 집계
- 항공사/월/출발공항코드 별 취소 총수 
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - CANCELLED: 취소여부
- 집계 : sum    

In [112]:
flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR'])['CANCELLED'].sum().to_frame()

CANCELLED
AIRLINE MONTH ORG_AIR           
AA      1     ATL              0
              DEN              0
              DFW              8
              IAH              0
              LAS              0
...                          ...
WN      12    LAS              1
              LAX              2
              MSP              0
              PHX              0
              SFO              0

[1133 rows x 1 columns]

In [114]:
flights.pivot_table(index=['AIRLINE', 'ORG_AIR'], columns='MONTH', values='CANCELLED', aggfunc="sum", margins=True)

MONTH                1      2     3     4     5      6     7     8     9  \
AIRLINE ORG_AIR                                                            
AA      ATL        0.0    2.0   1.0   0.0   0.0    0.0   0.0   0.0   0.0   
        DEN        0.0    1.0   0.0   0.0   2.0    0.0   0.0   0.0   0.0   
        DFW        8.0   33.0  13.0   4.0   8.0    7.0   1.0   2.0   1.0   
        IAH        0.0    1.0   0.0   0.0   1.0    0.0   0.0   1.0   0.0   
        LAS        0.0    2.0   0.0   0.0   0.0    0.0   1.0   0.0   0.0   
...                ...    ...   ...   ...   ...    ...   ...   ...   ...   
WN      LAX        3.0    2.0   3.0   2.0   1.0    0.0   9.0   4.0   3.0   
        MSP        1.0    0.0   0.0   0.0   0.0    0.0   0.0   0.0   0.0   
        PHX        0.0    2.0   1.0   1.0   0.0    0.0   1.0   0.0   0.0   
        SFO        4.0    5.0   0.0   2.0   2.0    6.0   0.0   2.0   2.0   
All              104.0  195.0  94.0  58.0  81.0  100.0  56.0  44.0  17.0   

MONTH              11    12  All  
AIRLINE ORG_AIR                   
AA      ATL       0.0   0.0    3  
        DEN       0.0   1.0    4  
        DFW       3.0   6.0   86  
        IAH       0.0   0.0    3  
        LAS       0.0   0.0    3  
...               ...   ...  ...  
WN      LAX       3.0   2.0   32  
        MSP       0.0   0.0    1  
        PHX       1.0   0.0    6  
        SFO       2.0   0.0   25  
All              51.0  81.0  881  

[115 rows x 12 columns]

## 3개 이상의 컬럼을 grouping해서 집계 2
- 항공사/월/출발공항코드 별 최대/최소 연착시간
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - ARR_DELAY: 연착시간
- 집계 : min, max    

In [117]:
flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR'])['ARR_DELAY'].agg(["min", "max"])

min    max
AIRLINE MONTH ORG_AIR             
AA      1     ATL     -27.0   26.0
              DEN     -13.0   78.0
              DFW     -39.0  287.0
              IAH     -23.0   63.0
              LAS     -32.0  732.0
...                     ...    ...
WN      12    LAS     -52.0   96.0
              LAX     -30.0  493.0
              MSP     -23.0   90.0
              PHX     -30.0  254.0
              SFO     -20.0  215.0

[1133 rows x 2 columns]

In [120]:
result = flights.pivot_table(index='MONTH', columns=['AIRLINE', 'ORG_AIR'], values='ARR_DELAY', aggfunc=['min', 'max'])
result

min                                                        ...  \
AIRLINE    AA                                                        ...   
ORG_AIR   ATL   DEN   DFW   IAH   LAS   LAX   MSP   ORD   PHX   SFO  ...   
MONTH                                                                ...   
1       -27.0 -13.0 -39.0 -23.0 -32.0 -42.0 -34.0 -38.0 -15.0 -20.0  ...   
2       -26.0 -17.0 -29.0 -27.0 -25.0 -41.0 -32.0 -34.0 -16.0 -31.0  ...   
3       -32.0 -19.0 -29.0 -13.0 -24.0 -31.0 -38.0 -34.0 -24.0 -31.0  ...   
4       -30.0 -28.0 -37.0 -27.0 -19.0 -40.0 -10.0 -52.0 -22.0 -33.0  ...   
5       -26.0 -20.0 -36.0 -19.0 -31.0 -41.0 -15.0 -32.0 -17.0 -45.0  ...   
6       -23.0 -13.0 -33.0 -13.0 -27.0 -39.0 -17.0 -43.0 -26.0 -33.0  ...   
7       -27.0 -30.0 -32.0 -19.0 -40.0 -42.0 -26.0 -35.0 -26.0 -46.0  ...   
8       -32.0 -27.0 -32.0 -30.0 -28.0 -49.0 -22.0 -36.0 -26.0 -39.0  ...   
9       -33.0 -27.0 -33.0 -31.0 -32.0 -38.0 -28.0 -46.0 -27.0 -38.0  ...   
11      -33.0 -27.0 -45.0 -14.0 -31.0 -37.0 -33.0 -50.0 -35.0 -29.0  ...   
12      -31.0 -40.0 -42.0 -15.0 -33.0 -45.0 -41.0 -41.0 -60.0 -37.0  ...   

           max                                                                 
AIRLINE     VX                   WN                                            
ORG_AIR    LAX    ORD    SFO    ATL    DEN    LAS    LAX    MSP    PHX    SFO  
MONTH                                                                          
1        105.0   58.0   87.0  126.0  131.0  189.0  179.0  128.0  114.0   25.0  
2         36.0   54.0   54.0  111.0  106.0  135.0  190.0   68.0  159.0  110.0  
3        224.0    NaN  156.0  140.0  375.0  261.0  104.0   43.0   78.0  237.0  
4         23.0  126.0   93.0  175.0  192.0  163.0  165.0   64.0  130.0   72.0  
5        109.0   51.0   99.0  198.0  261.0  150.0  244.0   13.0  203.0  133.0  
6        215.0    8.0  147.0  268.0  152.0  193.0  244.0   88.0  168.0  205.0  
7         44.0   10.0  177.0  184.0  235.0  290.0  248.0   84.0  171.0   51.0  
8        230.0   21.0  165.0  184.0  140.0  284.0  221.0   45.0  161.0   65.0  
9         92.0    6.0   46.0  134.0  125.0  284.0  157.0   23.0  131.0   69.0  
11        53.0  150.0   58.0   79.0  123.0  120.0   87.0  114.0  244.0  143.0  
12       115.0  159.0  236.0  187.0  160.0   96.0  493.0   90.0  254.0  215.0  

[11 rows x 228 columns]

In [121]:
result.to_excel('saved_data/result.xlsx')

# apply() - Series, DataFrame의 데이터 일괄 처리

데이터프레임의 행들과 열들 또는 Series의 원소들에 공통된 처리를 할 때 apply 함수를 이용하면 반복문을 사용하지 않고 일괄 처리가 가능하다.

- DataFrame.apply(함수, axis=0, args=(), \*\*kwarg)
    - 인수로 행이나 열을 받는 함수를 apply 메서드의 인수로 넣으면 데이터프레임의 행이나 열들을 하나씩 함수에 전달한다.
    - 매개변수
        - **함수**: DataFrame의 행들 또는 열들을 전달할 함수
        - **axis**: **0-컬럼(열)을 전달, 1-행을 전달 (기본값 0)**
        - **args**: 함수에 행/열 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달
        - **\*\*kwarg**: 함수에 행/열 이외에 전달할 매개변수를 키워드 인자로 전달
- Series.apply(함수, args=(), \*\*kwarg)
    - 인수로 Series의 원소들을 받는 함수를 apply 메소드의 인수로 넣으면  Series의 원소들을 하나씩 함수로 전달한다.
    - 매개변수
        - **함수**: Series의 원소들을 전달할 함수
        - **args**: 함수에 원소 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달
        - **\*\*kwarg**: 함수에 원소 이외에 전달할 매개변수를 키워드 인자로 전달

In [25]:
import numpy as np
import pandas as pd

a = np.arange(24).reshape(6, 4) 
# arange(24)  0 ~ 23 1씩 증가하는 점수로 구성된 1차원 배열
# reshape(6, 4) 배열의 형태(shape)를 변경 - 6 X 4 2차원 배열로 변환.
df = pd.DataFrame(a, columns=['no1', 'no2', 'no3', 'no4'])
df

,no1,no2,no3,no4
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23


In [35]:
def func1(x):
    """
    x: DataFrame.apply() - Series를 전달(열-axis=0/행-axis=1)
       Series.apply() - Series를 구성하는 원소 하나
    """
    if x.name in ['no1', 'no3']:
        return x * 100
    else:
        return x / 100

In [36]:
df.apply(func1) # 여러 값이 return 되면 DataFrame 형태로 출력

,no1,no2,no3,no4
0,0,0.01,200,0.03
1,400,0.05,600,0.07
2,800,0.09,1000,0.11
3,1200,0.13,1400,0.15
4,1600,0.17,1800,0.19
5,2000,0.21,2200,0.23


In [29]:
df['no1'].name # Series를 조회한 컬럼/index 이름

'no1'

In [30]:
df.iloc[3].name

3

In [39]:
df.apply(lambda x : x.max()-x.min()) # 단일한 값이 return 되면 Series 형태로 나옴

no1    20
no2    20
no3    20
no4    20
dtype: int32

In [40]:
def func2(x):
    print(x)
    print('--------------')
    return 1

In [41]:
df['no1'].apply(func2)  # Series

0
--------------
4
--------------
8
--------------
12
--------------
16
--------------
20
--------------


0    1
1    1
2    1
3    1
4    1
5    1
Name: no1, dtype: int64

In [42]:
df.apply(lambda x : x.mean() - x.median())  # 지금 상황에서는 모두 수치형이지만 수치형이 아닐 수도 있기 때문에 lamba를 사용해서는 안된다.

no1    0.0
no2    0.0
no3    0.0
no4    0.0
dtype: float64

# cut()/qcut() - 연속형(실수)을 범주형으로 변환

- cut() : 지정한 값을 기준으로 구간을 나눠 그룹으로 묶는다.
    - `pd.cut(x, bins,right=True, labels=None)`
    - 매개변수
        - **x**: 범주형으로 바꿀 대상. 1차원 배열형태(Series, 리스트, ndarray)의 자료구조
        - **bins**: 범주로 나눌때의 기준값(구간경계)들을 리스트로 묶어서 전달한다.
        - **right**: 구간경계의 오른쪽(True-기본)을 포함할지 왼쪽(False)을 포함할지
        - **labels**: 각 구간(범주)의 label을 리스트로 전달
            - 생략하면 범위를 범주명으로 사용한다. (ex: (10, 20], ()-포함안함, []-포함)
- qcut() :  대상배열의 최대값 ~ 최소값을 지정한 개수의 동등한 size(**원소의개수**)가 되도록 나눈다.
    - `pd.qcut(x, q, labels)`
    - 매개변수
        - **x**: 나눌 대상. 1차원 배열형태의 자료구조
        - **q**: 나눌 개수
        - **labels**: 각 구간(범주)의 label을 리스트로 전달

In [59]:
np.random.seed(0)
age = np.random.randint(1, 100, size=30) # 1~99 사이의 난수 30개
tall = np.round(np.random.normal(170, 10, size=30), 2)
# 평균: 170, 표준편차: 10인 정규분포를 따르는 난수 30개, 대부분의 난수가 170-10*2 ~ 170+10*2 범위, 평균에 가까운값들이 많이 생성.
print(age)
print(tall)

[45 48 65 68 68 10 84 22 37 88 71 89 89 13 59 66 40 88 47 89 82 38 26 78
 73 10 21 81 70 80]
[168.18 184.1  166.26 172.75 160.39 173.77 170.33 176.81 154.37 164.33
 167.58 185.14 166.67 170.47 184.63 185.35 175.66 171.49 159.22 183.95
 187.87 164.3  171.75 165.37 159.14 176.4  166.14 162.24 179.96 150.67]


In [61]:
df = pd.DataFrame({
    "나이":age,
    "키":tall
})
df

,나이,키
0,45,168.18
1,48,184.10
2,65,166.26
3,68,172.75
4,68,160.39
5,10,173.77
6,84,170.33
7,22,176.81
8,37,154.37
9,88,164.33


In [62]:
df.나이.value_counts()

89    3
68    2
10    2
88    2
45    1
47    1
70    1
81    1
21    1
73    1
78    1
26    1
38    1
82    1
59    1
40    1
66    1
48    1
13    1
71    1
37    1
22    1
84    1
65    1
80    1
Name: 나이, dtype: int64

In [63]:
df.키.value_counts() # 연속형이라서 중복되는 값이 없음

168.18    1
184.10    1
179.96    1
162.24    1
166.14    1
176.40    1
159.14    1
165.37    1
171.75    1
164.30    1
187.87    1
183.95    1
159.22    1
171.49    1
175.66    1
185.35    1
184.63    1
170.47    1
166.67    1
185.14    1
167.58    1
164.33    1
154.37    1
176.81    1
170.33    1
173.77    1
160.39    1
172.75    1
166.26    1
150.67    1
Name: 키, dtype: int64

In [71]:
# 연속형 값 -> 범주형 값 변환 (범위로 묶어서 같은 범주로 만든다.) ==> cut()/qcut()
# cut
나이대 = pd.cut(df['나이'], bins=3) # df['나이'] ==> 같은 범위로 3등분
나이대.value_counts()  
# [] - closed(포함 O), () - opened(포함 X)
# (36.333, 62.667] =>  36.333 < x <= 62.667

(62.667, 89.0]      17
(36.333, 62.667]     7
(9.921, 36.333]      6
Name: 나이, dtype: int64

In [72]:
df['나이'].agg(['min', 'max']) # min 값은 포함 X 니까 10이 아니라 9.921로 값을 나눔/ 반대로 max 값은 포함하니까 89로 값을 나눔

min    10
max    89
Name: 나이, dtype: int32

In [73]:
나이대 = pd.cut(df['나이'], bins=3, right=False) # right=False: 작은 쪽 값를 포함 default: right=True
나이대.value_counts()  

[62.667, 89.079)    17
[36.333, 62.667)     7
[10.0, 36.333)       6
Name: 나이, dtype: int64

In [76]:
나이대 = pd.cut(df['나이'], bins=3, right=False, labels=['나이대1', '나이대2', '나이대3']) # 등분된 값에 이름 붙이기
나이대

0     나이대2
1     나이대2
2     나이대3
3     나이대3
4     나이대3
5     나이대1
6     나이대3
7     나이대1
8     나이대2
9     나이대3
10    나이대3
11    나이대3
12    나이대3
13    나이대1
14    나이대2
15    나이대3
16    나이대2
17    나이대3
18    나이대2
19    나이대3
20    나이대3
21    나이대2
22    나이대1
23    나이대3
24    나이대3
25    나이대1
26    나이대1
27    나이대3
28    나이대3
29    나이대3
Name: 나이, dtype: category
Categories (3, object): ['나이대1' < '나이대2' < '나이대3']

In [77]:
df['나이대'] = 나이대
df

,나이,키,나이대
0,45,168.18,나이대2
1,48,184.10,나이대2
2,65,166.26,나이대3
3,68,172.75,나이대3
4,68,160.39,나이대3
5,10,173.77,나이대1
6,84,170.33,나이대3
7,22,176.81,나이대1
8,37,154.37,나이대2
9,88,164.33,나이대3


In [80]:
l = [0, 20, 40, 60, 90]
나이대2 = pd.cut(df['나이'], bins=4, labels=['청소년', '청년', '장년', '노년'])
나이대2

0      청년
1      청년
2      장년
3      장년
4      장년
5     청소년
6      노년
7     청소년
8      청년
9      노년
10     노년
11     노년
12     노년
13    청소년
14     장년
15     장년
16     청년
17     노년
18     청년
19     노년
20     노년
21     청년
22    청소년
23     노년
24     노년
25    청소년
26    청소년
27     노년
28     노년
29     노년
Name: 나이, dtype: category
Categories (4, object): ['청소년' < '청년' < '장년' < '노년']

In [81]:
df['나이대2'] = 나이대2
df

,나이,키,나이대,나이대2
0,45,168.18,나이대2,청년
1,48,184.10,나이대2,청년
2,65,166.26,나이대3,장년
3,68,172.75,나이대3,장년
4,68,160.39,나이대3,장년
5,10,173.77,나이대1,청소년
6,84,170.33,나이대3,노년
7,22,176.81,나이대1,청소년
8,37,154.37,나이대2,청년
9,88,164.33,나이대3,노년


In [83]:
df['나이대2'].value_counts()

노년     13
청소년     6
청년      6
장년      5
Name: 나이대2, dtype: int64

In [82]:
df['나이대2'].value_counts(normalize=True)

노년     0.433333
청소년    0.200000
청년     0.200000
장년     0.166667
Name: 나이대2, dtype: float64

In [84]:
df.groupby('나이대2')['키'].mean()

나이대2
청소년    172.556667
청년     167.638333
장년     173.876000
노년     170.364615
Name: 키, dtype: float64

In [87]:
# qcut # 동일한 원소수(데이터 개수)로 나눈다.
키대 = pd.qcut(df['키'], # 나눌 대상
             q=3,       # 몇개로 나눌지
             labels=['작은키', '중간키', '큰키']
           )
키대.value_counts()

작은키    10
중간키    10
큰키     10
Name: 키, dtype: int64

In [88]:
df['키대'] = 키대
df

,나이,키,나이대,나이대2,키대
0,45,168.18,나이대2,청년,중간키
1,48,184.10,나이대2,청년,큰키
2,65,166.26,나이대3,장년,중간키
3,68,172.75,나이대3,장년,중간키
4,68,160.39,나이대3,장년,작은키
5,10,173.77,나이대1,청소년,중간키
6,84,170.33,나이대3,노년,중간키
7,22,176.81,나이대1,청소년,큰키
8,37,154.37,나이대2,청년,작은키
9,88,164.33,나이대3,노년,작은키


In [89]:
df.groupby('키대')['나이'].mean()

키대
작은키    61.1
중간키    55.9
큰키     57.5
Name: 나이, dtype: float64

<b style='font-size:2em'>TODO </b>

In [90]:
# 1: data/diamonds.csv 를 읽어 DataFrame으로 만든다.
import pandas as pd

dia = pd.read_csv('data/diamonds.csv')
dia

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [93]:
# 2: price 컬럼을 '고가', '중가', '저가' 세개의 범주값을 가지는 "price_cate" 컬럼을 생성한다.
price_cate = pd.cut(dia['price'], bins=3,labels=['고가', '중가', '저가'])
dia['price_cate'] = price_cate
dia

,carat,cut,color,clarity,depth,table,price,x,y,z,price_cate
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,고가
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,고가
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,고가
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,고가
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,고가
...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50,고가
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61,고가
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56,고가
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74,고가


In [94]:
# 3 가격대(price_cate) 별 carat의 평균을 조회
dia.groupby('price_cate')['carat'].mean()

price_cate
고가    0.627943
중가    1.362280
저가    1.885256
Name: carat, dtype: float64

In [96]:
# 4 가격대(price_cate)와 cut별 평균 가격(price)를 피봇테이블로 조회
dia.pivot_table(index='price_cate', columns='cut', values='price')

cut,Fair,Good,Ideal,Premium,Very Good
price_cate,,,,,
고가,2976.492819,2589.423161,1991.838539,2513.585759,2383.723178
중가,8891.044335,8744.889273,9019.390308,9045.209888,8921.683740
저가,15176.404762,15412.570213,15422.528676,15391.247016,15502.217323


In [99]:
# 5 cut, color, price_cate 별 carat의 평균을 피봇테이블로 조회
dia.pivot_table(index=['price_cate','color'], columns='cut', values='carat')

cut                   Fair      Good     Ideal   Premium  Very Good
price_cate color                                                   
고가         D      0.807122  0.670725  0.502135  0.599525   0.599320
           E      0.767424  0.651305  0.511158  0.591792   0.577348
           F      0.791801  0.689913  0.529869  0.666792   0.619161
           G      0.849462  0.708228  0.528951  0.629871   0.614364
           H      0.977699  0.730387  0.617462  0.765683   0.712640
           I      0.988750  0.764122  0.669981  0.794620   0.760677
           J      1.067717  0.854693  0.792335  0.901903   0.832754
중가         D      1.339412  1.277551  1.170843  1.264649   1.226980
           E      1.443500  1.256889  1.198333  1.260486   1.212459
           F      1.564483  1.298312  1.189853  1.268342   1.193546
           G      1.807895  1.349915  1.193722  1.300076   1.244620
           H      1.796226  1.523187  1.398540  1.525855   1.489892
           I      1.784000  1.630000  1.533642  1.610519   1.551324
           J      1.985000  1.695397  1.645655  1.716263   1.707778
저가         D      2.145714  1.626500  1.430909  1.636000   1.552857
           E      1.843333  1.717097  1.501395  1.715130   1.740141
           F      1.957273  1.809286  1.609551  1.705594   1.688632
           G      1.995000  1.864048  1.694875  1.793261   1.735664
           H      2.218750  2.093810  1.947143  2.069171   2.019664
           I      2.407778  2.104643  2.097365  2.127762   2.100703
           J      2.691818  2.242500  2.229880  2.236724   2.150377